Het maken van de dif_batches
-
Om met behulp van PCA anomalieën te detecteren in peniciline productie worden er vier stappen gezet.

1. Het maken van een NOC_batch
2. Het maken van de dif_batches
3. Het uitvoeren van de PCA
4. Het classificeren van de data

Deze stappen komen rechtstreeks uit een eerder opgesteld document om de werking van PCA uit te leggen en hoe PCA toe te passen in ons project. Dit document is meegestuurd in de bijlage.

In dit jupyter notebook wordt de derde stap uitgevoerd, het uitvoeren van een PCA. Op basis van de dif_batches of de pooled_batches wordt doormiddel van de PCA teruggewerkt naar drie principal components zodat de data geplot kan worden.

In dit geval is er gebruik gemaakt van een pooled_batches dataframe. Hierin staan de dif_batches gepooled in pools van 20 timestamps. Meer informatie over deze dataframe staat in de jupyter notebook 'pooled_batches'. 

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
plt.rcParams['figure.figsize'] = [25, 5]

pooled_batches = pd.read_csv(r'pooled_batches_20.csv')


Hieronder wordt het dataframe geprint. In de dataframe zitten 86 batches die ieder ongeveer 50 pools bevatten. 

In [10]:
pooled_batches

,Aeration rate(Fg:L/h),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),Dumped broth flow(Fremoved:L/h),Substrate concentration(S:g/L),...,Generated heat(Q:kJ),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oil flow(Foil:L/hr),Oxygen Uptake Rate(OUR:(g min^{-1})),Carbon evolution rate(CER:g/h),Ammonia shots(NH3_shots:kgs),Batch_id,Timestamp,Faulty
0,0.0,0.0,4.906304,238.571317,500.534273,506.546440,0.0,2.220446e-15,0.0,0.975620,...,5787.495292,0.027719,0.000000,0.0,0.842884,0.010536,0.0,1.0,20.0,False
1,0.0,0.0,0.000000,157.845736,150.154679,4.235639,0.0,2.220446e-15,0.0,1.116864,...,6177.755671,0.025026,0.000000,0.0,0.267870,0.009534,0.0,1.0,40.0,False
2,0.0,0.0,0.444484,241.161600,740.100214,77.168623,0.0,2.220446e-15,0.0,1.496355,...,6500.554242,0.135286,0.000000,0.0,1.424782,0.072152,0.0,1.0,60.0,False
3,0.0,0.0,0.556081,294.064500,1670.234786,491.573895,0.0,2.220446e-16,0.0,3.224266,...,5820.736726,0.129764,0.000000,0.0,1.972338,0.069250,0.0,1.0,80.0,False
4,0.0,0.0,0.111287,750.622643,2333.650357,82.388586,0.0,0.000000e+00,0.0,0.095226,...,5901.126143,0.555931,0.000000,0.0,2.787246,0.296856,0.0,1.0,100.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,0.0,0.0,0.000000,1196.689486,507.313802,848.203797,0.0,4.440892e-15,0.0,1050.596005,...,888.675571,5.751943,38.370829,0.0,11.804550,5.514436,0.0,100.0,920.0,True
4268,0.0,0.0,0.000000,1168.727657,518.306181,786.909263,0.0,4.440892e-15,0.0,1091.870130,...,787.526286,5.949650,38.056557,0.0,11.574139,5.703707,0.0,100.0,940.0,True
4269,0.0,0.0,14.340592,906.293870,521.778182,880.711718,0.0,4.440892e-15,0.0,1135.677933,...,703.240857,6.052743,37.325971,0.0,13.667070,5.801200,0.0,100.0,960.0,True
4270,0.0,0.0,19.817549,1217.744737,557.600130,856.184106,0.0,4.440892e-15,0.0,1181.287104,...,835.557429,6.011971,47.212393,0.0,13.447560,5.758729,0.0,100.0,980.0,True


In de cel hieronder wordt de PCA uitgevoerd. 

Eerst wordt de data gescaled, dit wordt gedaan met de standardscaler van sklearn. Hier is voor gekozen omdat de data veel extreme punten bevat en door gebruik te maken van een standardscaler wordt de data minder opgekropt.

Hierna wordt de PCA uitgevoerd. Er wordt, zoals eerder gesteld, naar drie componenten toegewerkt. Dit is enkel voor het plotten van de data en voor het classificeren wordt weer de voledige data gebruikt. 

In [11]:
scaler = StandardScaler()

scaler.fit(pooled_batches.iloc[:,:len(pooled_batches.columns)-3])
input_pca = scaler.transform(pooled_batches.iloc[:,:len(pooled_batches.columns)-3])

pca = PCA(n_components=3)
pca_data = pca.fit_transform(input_pca)
pca_data = pd.DataFrame(pca_data, columns = ['PC1','PC2','PC3'])
pca_data['Timestamp'] = list(pooled_batches["Timestamp"])
pca_data['Faulty'] = list(pooled_batches["Faulty"])
pca_data

,PC1,PC2,PC3,Timestamp,Faulty
0,-1.341034,0.873575,0.250926,20.0,False
1,-2.037416,0.099128,0.224634,40.0,False
2,-1.739731,0.579993,0.379528,60.0,False
3,-1.423277,1.476860,0.808317,80.0,False
4,-1.308867,1.164782,1.154634,100.0,False
...,...,...,...,...,...
4267,7.297498,-0.593974,-1.687029,920.0,True
4268,7.424592,-0.726171,-1.704388,940.0,True
4269,8.417775,-0.676031,-1.501203,960.0,True
4270,8.982560,-0.530985,-1.463084,980.0,True


In [12]:
xlim = (min(pca_data['PC1']),max(pca_data['PC1']))
ylim = (min(pca_data['PC2']),max(pca_data['PC2']))
zlim = (min(pca_data['PC3']),max(pca_data['PC3']))

Nu dat we de PCA hebben uitgevoerd gaan we de data plotten. Er wordt een plot gemaakt op elke timestamp of in dit geval op elke 20 timestamps omdat dit de pool grote is. Elk plot wordt opgeslagen zodat hier later naar gekeken kan worden. Er is ook een korte code geschreven om automatisch van de plots een geanimeerde gif te maken. 

In [13]:
for ii in range(20,1001,20):
    fig = plt.figure()

    ax = fig.add_subplot(projection='3d')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)
    for i in range(len(pca_data)):
        if pca_data.iloc[i,3] == ii:
            if pca_data.iloc[i,4]:
                ax.scatter(pca_data.iloc[i,0],pca_data.iloc[i,1],pca_data.iloc[i,2], c='r', alpha=0.5)
            else:
                ax.scatter(pca_data.iloc[i,0],pca_data.iloc[i,1],pca_data.iloc[i,2], c='g', alpha=0.3)

    
    #plt.savefig(str(ii)+'.png', bbox_inches='tight')
    plt.close()